# **Download Data**


If the Google drive links are dead, you can download data from [kaggle](https://www.kaggle.com/naebolo/predict-traffic-of-lte-network?select=test.csv), and upload data manually to the workspace.

In [21]:
# Create a folder where the dataset will be stored
!git clone https://github.com/lirits/NetworkTrafficPrediction.git
!mkdir NetworkTrafficPrediction/Dataset -p
!gdown --id '18WeCp7OPGulOgMl-KN6DN1CJw47IlTQ2' --output NetworkTrafficPrediction/Dataset/lte.train.csv
# !gdown --id '18_nW3nufgDzFFmpuPAWyKUbO11gLU8Ud' --output NetworkTrafficPrediction/Dataset/lte.test.csv # We have abandoned it because of the insufficient amount of data.

fatal: destination path 'NetworkTrafficPrediction' already exists and is not an empty directory.
Downloading...
From: https://drive.google.com/uc?id=18WeCp7OPGulOgMl-KN6DN1CJw47IlTQ2
To: /content/NetworkTrafficPrediction/Dataset/lte.train.csv
100% 20.0M/20.0M [00:00<00:00, 122MB/s] 


# Import Some packages

In [27]:
import sys
if not 'NetworkTrafficPrediction' in sys.path:
    sys.path += ['NetworkTrafficPrediction']

In [29]:
# pytorch
import torch
import torch.nn as nn

from data.custom_dataset import LteTrafficDataloader


# For data plotting
import matplotlib.pyplot as plt

# seed
torch.backends.cudnn.deterministic = True
torch.manual_seed(hash("by removing stochasticity") % 2**32 - 1)
torch.cuda.manual_seed_all(hash("so runs are repeatable") % 2**32 - 1)

ImportError: ignored

## **Dataloader**
A `DataLoader` loads data from a given `Dataset` into batches.


In [24]:
train_file_path = 'NetworkTrafficPrediction/Dataset/lte.train.csv'
CellName = 'Cell_003781'
batch_size = 128
windows_size = 168
target_size = 48
learning_rate = 1e-3
epochs =100
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [28]:
# Creating data indices for training and validation splits:
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np
dataset = LteTrafficDataset(train_file_path,CellName=CellName,windows_size=windows_size,target_size=target_size)
validation_split = .2
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
train_indices, val_indices = indices[split:], indices[:split]
train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                           sampler=SubsetRandomSampler(train_indices))
validation_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                                sampler=SubsetRandomSampler(val_indices))

In [32]:
example_x,example_y = next(iter(train_loader))
print(example_x.shape,example_y.shape)

torch.Size([128, 168, 1]) torch.Size([128, 48, 1])


# Define model

In [8]:
moid = 'RNN'
hidden_size = 200
num_lstm = 1
dropout = 0.2
bidirectional = False
net = LSTMModule(moid,windows_size,1,target_size,hidden_size,num_lstm,dropout,bidirectional).to(device)
print(net)

LSTMModule(
  (RNNS_model): Sequential(
    (0): LSTM(1, 200)
  )
  (fc1): Flatten(start_dim=1, end_dim=-1)
  (linear_model): Sequential(
    (0): Dropout(p=0.2, inplace=False)
    (1): Linear(in_features=33600, out_features=16800, bias=True)
    (2): ReLU()
    (3): Linear(in_features=16800, out_features=48, bias=True)
  )
)


In [10]:
pred_y = net(example_x.to(device))
print(example_x.shape,example_y.shape,pred_y.shape)

torch.Size([128, 168, 1]) torch.Size([128, 48, 1]) torch.Size([128, 48, 1])


# Train loop

In [11]:
from utils.tools import train_loop

In [12]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(),lr=learning_rate)

In [16]:
def train_loop(
        datalodaer,
        model,
        loss_fn,
        optimizer,
        device,
        usd_wandb: bool = False,
        use_Tr_accuracy: bool = False,
        tolerate_rate:float = 0.1):
    size = len(datalodaer.dataset)
    accuracy = 0
    for batch, (X, y) in enumerate(datalodaer):
        X, y = X.to(device), y.to(device)
        # compute prediction and loss
        pred = model(X)
        # loss = torch.sqrt(loss_fn(pred, y))
        loss = loss_fn(pred, y)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if use_Tr_accuracy:
            accuracy += Tr_accuracy(pred, y,tolerate_rate)

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            if usd_wandb:
                import wandb
                wandb.log({'train_loss': loss.item(), 'Tr_accuracy': accuracy})
            print(
                f"loss: {loss:>7f},Accuracy:{(accuracy/batch)}  [{current:>5d}/{size:>5d}]")

In [18]:
def Tr_accuracy(y_pred: float, y_true: float, tolerate_rate: int) -> float:
    return ((y_true * (1 - tolerate_rate) < y_pred) == (y_pred < (1 + tolerate_rate)
            * y_true)).sum() / (y_pred.shape[0] * y_pred.shape[1] * y_pred.shape[2])

In [19]:
for t in range(epochs):
  print(f'Epoch {t+1}\n-------------------------------------')
  train_loop(train_dataloader,net,loss_fn,optimizer,device,False,True)

Epoch 1
-------------------------------------
loss: 0.217976,Accuracy:nan  [    0/ 8518]
Epoch 2
-------------------------------------
loss: 0.008422,Accuracy:inf  [    0/ 8518]
Epoch 3
-------------------------------------
loss: 0.007658,Accuracy:inf  [    0/ 8518]
Epoch 4
-------------------------------------
loss: 0.007206,Accuracy:inf  [    0/ 8518]
Epoch 5
-------------------------------------
loss: 0.007530,Accuracy:inf  [    0/ 8518]
Epoch 6
-------------------------------------
loss: 0.007680,Accuracy:inf  [    0/ 8518]
Epoch 7
-------------------------------------
loss: 0.006996,Accuracy:inf  [    0/ 8518]
Epoch 8
-------------------------------------
loss: 0.006768,Accuracy:inf  [    0/ 8518]
Epoch 9
-------------------------------------
loss: 0.006251,Accuracy:inf  [    0/ 8518]
Epoch 10
-------------------------------------
loss: 0.007559,Accuracy:inf  [    0/ 8518]
Epoch 11
-------------------------------------
loss: 0.007227,Accuracy:inf  [    0/ 8518]
Epoch 12
----------

In [33]:
!nvidia-smi

Wed Oct 20 17:06:25 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0    35W / 250W |  14263MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [24]:
len(training_data)

2